In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu


In [7]:
!ls

citylearn-2022	nvidia-acknowledgements  sustaingym.code-workspace
mosek		sustaingym


In [8]:
import os

import numpy as np
import seaborn as sns
import stable_baselines3 as sb3
from tqdm.auto import tqdm

from sustaingym.envs import ElectricityMarketEnv, CongestedElectricityMarketEnv
from sustaingym.envs.battery.plot_utils import *
from sustaingym.envs.battery.wrapped import DiscreteActions
from sustaingym.evaluate.run_electricitymarket import *


sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

In [10]:
from stable_baselines3.common.env_checker import check_env

env = CongestedElectricityMarketEnv(congestion=False)

env.reset(seed=1)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Mar 27 05:02:53 PM: Your problem has 1220 variables, 9 constraints, and 2506 parameters.
(CVXPY) Mar 27 05:02:53 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 27 05:02:53 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 27 05:02:53 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Mar 27 05:02:53 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) Mar 27 05:02:53 PM: Applying reduction Dcp2Cone
(CVXPY) Mar 27 05:02:53 PM: Applying reduction CvxAttr2Constr
(

TypeError: bad operand type for unary -: 'NoneType'

In [7]:
%debug

> /home/ubuntu/.conda/envs/sustaingym/lib/python3.9/site-packages/mosek/__init__.py(10973)putnaintparam()
  10971       _arg1_paramname_ = _arg1_paramname
  10972       _arg2_parvalue_ = _arg2_parvalue
> 10973       res = self.__obj.putnaintparam(_arg1_paramname_,_arg2_parvalue_)
  10974       if res != 0:
  10975         result,msg = self.__getlasterror(res)



In [5]:
from sustaingym.envs import CongestedMarketOperator

env = CongestedElectricityMarketEnv(congestion=False)
market_op = CongestedMarketOperator(env)

AttributeError: 'CongestedElectricityMarketEnv' object has no attribute 'demand'

In [32]:
market_op.out.value = np.zeros((env.settlement_interval + 1, env.network.D+env.network.G+env.network.L+env.network.L))

market_op.cgen.value = env.network.cgen


market_op.prob.solve(solver='MOSEK')

print("obj: ", market_op.prob.value)

ParameterError: A Parameter (whose name is 'generator production costs') does not have a value associated with it; all Parameter objects must have values before solving a problem.

In [6]:
%debug

> /home/ubuntu/.conda/envs/sustaingym/lib/python3.9/site-packages/numpy/core/numeric.py(2355)isclose()
   2353         y = asanyarray(y, dtype=dt)
   2354 
-> 2355     xfin = isfinite(x)
   2356     yfin = isfinite(y)
   2357     if all(xfin) and all(yfin):

*** NameError: name 'x_bat_d' is not defined
*** NameError: name 'x_bat_d' is not defined
*** The 'jump' command requires a line number
*** Jump failed: f_lineno can only be set by a trace function
  /tmp/ipykernel_19072/1748134613.py(5)<module>()
      1 from stable_baselines3.common.env_checker import check_env
      2 
      3 env = CongestedElectricityMarketEnv(congestion=False)
      4 
----> 5 env.reset()

  /home/ubuntu/sustaingym/sustaingym/envs/battery/congested_electricity_market.py(871)reset()
    869         # print(self.demand_forecast)
    870 
--> 871         self.price = np.array([self._calculate_dispatch_without_agent(self.count)[3]], dtype=np.float32)
    872 
    873         print("action shape: ", self.action.sh

In [37]:
env = CongestedElectricityMarketEnv(congestion=False)
env.reset()

17
[1011.9403]
prob.status = infeasible
--Return--
None
> /home/ubuntu/sustaingym/sustaingym/envs/utils.py(17)solve_mosek()
     13         print(f'prob.status = {prob.status}')
     14         if 'infeasible' in prob.status:
     15             # your problem should never be infeasible. So now go debug
     16             import pdb
---> 17             pdb.set_trace()



TypeError: bad operand type for unary -: 'NoneType'

<h2>Creating network with battery<h2>

In [5]:
net = pn.case24_ieee_rts()
pp.create_storage(net, 23, p_mw = -30, max_e_mwh = 60, soc_percent = 1.0, min_e_mwh = 5)
# print(net['storage'])
print(net["load"])

    bus  const_i_percent  const_z_percent  controllable  in_service  name  \
0     0              0.0              0.0         False        True  None   
1     1              0.0              0.0         False        True  None   
10   12              0.0              0.0         False        True  None   
11   13              0.0              0.0         False        True  None   
12   14              0.0              0.0         False        True  None   
13   15              0.0              0.0         False        True  None   
14   17              0.0              0.0         False        True  None   
15   18              0.0              0.0         False        True  None   
16   19              0.0              0.0         False        True  None   
2     2              0.0              0.0         False        True  None   
3     3              0.0              0.0         False        True  None   
4     4              0.0              0.0         False        True  None   

<h2>Running DC OPF<h2>

In [6]:
# run optimal powerflow problem
pp.runopp(net, delta=1e-16)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



In [7]:
net.res_ext_grid

,p_mw,q_mvar
0,69.124879,68.856236


In [8]:
net.res_line.loading_percent # ensure no overloaded lins (greater than 100%)

0     16.323085
1     12.681878
10    29.557646
11    21.504352
12    17.886273
13    34.690820
14    13.338699
15    43.955822
16    41.981045
17    72.213067
18    21.804291
19    41.870690
2     37.127683
20    41.870690
21    37.297931
22    61.713411
23    18.799536
24    35.087727
25    27.170716
26    11.346060
27    11.346060
28     8.507202
29     8.507202
3     26.362726
30    20.624893
31    20.624893
32    30.424864
4     29.531863
5     18.424346
6     17.019500
7      8.605731
8     89.471856
9     50.489804
Name: loading_percent, dtype: float64

In [9]:
net.res_cost # how is this cost determined?

61906.897200933054

In [13]:
net.res_bus # power sink and injections
print(sum(net.res_bus['p_mw'])) # does not seem to match the amount of power received from external grid

-47.67038659507648


In [15]:
net.res_line # power across lines in opf

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,11.733273,-27.603449,-11.729914,-23.214862,0.003359,-50.818311,0.119509,0.103636,0.119509,1.050000,-6.451329,1.050000,-6.539039,16.323085
1,-3.955333,16.782165,4.159896,-22.073805,0.204564,-5.291641,0.068700,0.092850,0.092850,1.050000,-6.451329,1.012124,-5.414061,12.681878
10,-50.904013,9.094595,52.030809,-9.377409,1.126797,-0.282813,0.213043,0.216405,0.216405,1.015473,-12.431178,1.022101,-7.517082,29.557646
11,-36.033333,-12.727060,36.615964,10.215847,0.582631,-2.511213,0.157443,0.151616,0.157443,1.015473,-12.431178,1.048971,-9.469294,21.504352
12,-89.623099,7.586952,90.103889,-14.225901,0.480790,-6.638948,0.221413,0.224492,0.224492,1.019715,-2.393658,1.020000,0.000000,17.886273
13,-172.863441,-37.450473,174.471410,40.531240,1.607970,3.080767,0.435408,0.430412,0.435408,1.019715,-2.393658,1.044636,1.400107,34.690820
14,-66.103870,-13.497239,66.368532,5.254022,0.264662,-8.243217,0.167415,0.163844,0.167415,1.011609,-1.719058,1.020000,0.000000,13.338699
15,-222.328476,0.961764,228.333525,24.242092,6.005048,25.203856,0.551694,0.548941,0.551694,1.011609,-1.719058,1.050000,10.023802,43.955822
16,-214.097787,1.540590,219.001120,17.191113,4.903334,18.731703,0.526908,0.525173,0.526908,1.020000,0.000000,1.050000,10.023802,41.981045
17,-368.471410,74.421971,374.977348,-32.713875,6.505938,41.708095,0.903301,0.906353,0.906353,1.044636,1.400107,1.042475,9.172726,72.213067
